In [1]:
import os 
import shutil
import json
import numpy as np
import PIL.Image
from typing import List
import zipfile
from zipfile import ZipFile
from tqdm import tqdm
from PIL import Image



# move images to goal directory 
goal_dir = "/mnt/damian/Projects/minRF/data/car_models_3887/captions"
os.makedirs(goal_dir, exist_ok=True)
base_dir = "/mnt/damian/Projects/minRF/data/car_models_3887/train"
all_renders = os.listdir(base_dir)

# Captioning

In [2]:
import torch
from transformers import pipeline

# pipe = pipeline(
#     "image-text-to-text",
#     model= "google/gemma-3-27b-it",  # "google/gemma-3-4b-it", "google/gemma-3-12b-it", "google/gemma-3-27b-it" 
#     device_map="auto",
#     torch_dtype=torch.bfloat16
# )
pipe = pipeline("image-text-to-text", model="unsloth/gemma-3-27b-it-bnb-4bit", device_map="auto", torch_dtype=torch.bfloat16)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
Device set to use cuda:0


In [3]:

system_prompt = "You are an expert automotive design consultant with experience in 3D rendering analysis. Your task is to create detailed, technical descriptions of rendered 3D vehicle models shown in the image. \n \
                 You get an additional description of the car which you should also use for an accurate description of the vehicle or part of the vehicle in the image. Focus on: \n \
                 1. DESIGN SPECIFICS: Describe the vehicle's key design elements including body shape, distinctive features, surface treatments, and unique styling cues. \n \
                 2. VIEWPOINT & PERSPECTIVE: Precisely identify the camera angle (front/rear/side/3-4 view, high/low angle) to ensure spatial consistency. \n \
                 3. DETAILS & MATERIALS: Note important visual elements like wheels, lighting systems, surface finishes (metallic, matte, glossy), and material treatments (carbon fiber, chrome, etc.). \n \
                 4. COLOR & LIGHTING: Describe the vehicle's color scheme and how studio lighting interacts with the surfaces. \n \
                 5. BRAND IDENTITY: When recognizable, reference design language using brand terminology (e.g., \"kidney grille\" for BMW) or compare to iconic models (\"reminiscent of E46 proportions\"). \n \n \
                 FORMAT REQUIREMENTS: \n \
                 - Maintain technical precision while being descriptive  \n \
                 - Limit to 300 characters  \n \
                 - Avoid introductory phrases and meta-commentary  \n \
                 - Structure as a single, flowing description  \n \
                 - Emphasize elements that would be important for a diffusion model to reproduce accurately \n \n \
                 Your description should enable a text-to-image model to generate a faithful reproduction of the original image while maintaining the specific design intent and technical accuracy."


def create_image_caption(image_path, text, pipe):
    """Generate a summary for a single caption using the Gemma model"""
    # Load and process the image
    
    messages = [
                {
            "role": "system",
           "content": [{"type": "text", "text": system_prompt}]
        },
    {
        "role": "user",
        "content": [
            {"type": "image", "path": image_path},
            {"type": "text", "text": f"Additional description: {text} \n Please provide a detailed, technical description of the vehicle in the image for a text-to-image model."}
        ]
    }
    ]
    
    output = pipe(text=messages, max_new_tokens=300)
    summary = output[0]["generated_text"][-1]["content"]
    
    return summary

In [4]:
import pandas as pd 

train_prompts_path = "car_models_3887/train_prompts.csv"
train_prompts = pd.read_csv(train_prompts_path)
train_prompts.head()

,filename_image,caption
0,GMC_Sierra_1500_Extended_Cab_20072013_GMCSierr...,GMC GMC Sierra 1500 Extended Cab Truck The thi...
1,GMC_Sierra_1500_Extended_Cab_20072013_GMCSierr...,GMC GMC Sierra 1500 Extended Cab Truck The thi...
2,GMC_Sierra_1500_Extended_Cab_20072013_GMC-Sier...,GMC GMC Sierra 1500 Extended Cab Truck The thi...
3,GMC_Sierra_1500_Extended_Cab_20072013_GMCSierr...,GMC GMC Sierra 1500 Extended Cab Truck The thi...
4,GMC_Sierra_1500_Extended_Cab_20072013_GMC-Sier...,GMC GMC Sierra 1500 Extended Cab Truck The thi...


In [6]:
text_of_example = train_prompts.iloc[0]["caption"]
filename = train_prompts.iloc[0]["filename_image"]
image_path = os.path.join(base_dir, filename)
print(image_path)
print(text_of_example)

/mnt/damian/Projects/minRF/data/car_models_3887/train/GMC_Sierra_1500_Extended_Cab_20072013_GMCSierra1500ExtendedCab-3959_8.jpg
GMC GMC Sierra 1500 Extended Cab Truck The third generation of the Sierra was introduced in 2006 as a 2007 model, and the light-duty trucks appeared on the market in 2006.With over a century of experience on the market, GMC Trucks already knew what its customers want from their pickup-trucks. Even though it couldn't beat the Ford in the sales figures, that didn't mean that there was something wrong with the way the vehicle was built. It was more like the way the vehicle was priced. The T900 platform offered better solutions and more powerful engines than its predecessor, the T800.Like its predecessor, the Sierra was available in a few body versions, starting with a chassis-cab and going up to the big crew-cab version. Its raked windshield at 57 degrees decreased the wind noise and the aerodynamic resistance, increasing the fuel-efficiency. To add a more dynami

In [7]:
caption = create_image_caption(image_path, text_of_example, pipe)
print(caption)

Beige leather interior detail, GMC Sierra 1500 Extended Cab. Center console armrest, dual-tone beige/black. Integrated cupholders (x2) & slot. Soft-touch leather upper, textured black plastic lower. Red seatbelt buckle visible. Low-angle, close-up view. Matte finish leather, diffuse studio lighting. Focus on panel contours & material transitions. Crew cab interior, emphasizing practicality & robust construction.






In [10]:
caption

filename_image    GMC_Sierra_1500_Extended_Cab_20072013_GMCSierr...
caption           GMC GMC Sierra 1500 Extended Cab Truck The thi...
Name: 0, dtype: object

In [ ]:
# let's go through all the images and create captions for them
# save the caption in a json file in the goal_dir with the image name as the key  and the caption as the value
# check if the caption already exists, if it does, skip it
captions = {}
for filename, caption in tqdm(zip(train_prompts["filename_image"].to_list(), train_prompts["caption"].to_list()), total=len(train_prompts)):
    image_path = os.path.join(base_dir, filename)
    if not os.path.exists(image_path):
        continue
    filename_base = os.path.splitext(filename)[0]
    json_path = os.path.join(goal_dir, f"{filename_base}.json")
    if os.path.exists(json_path):
        with open(json_path, "r") as f:
            captions[filename] = json.load(f)
        continue

    if pd.isna(caption):
        caption = ""
    new_caption = create_image_caption(image_path, caption, pipe)
    
    # save the caption in a json file in the goal_dir with the image name as the key  and the caption as the value
    captions[filename] = caption
    with open(json_path, "w") as f:
        json.dump({filename: new_caption}, f, indent=4)
# save the captions in a csv file
captions_df = pd.DataFrame.from_dict(captions, orient='index', columns=['caption'])
captions_df.index.name = 'filename_image'
captions_df.reset_index(inplace=True)
captions_df.to_csv(os.path.join(goal_dir, "captions.csv"), index=False)

  2%|▏         | 3793/164176 [7:35:21<312:56:00,  7.02s/it]

In [ ]:
import os 

base_render_dir = "/mnt/damian/.objaverse/CarCaptionData_ai_toolkit/" #  "/mnt/damian/Projects/diffusers_finetune/renders_eval/"
object_directories = [f"{base_render_dir}/{object_id}" for object_id in os.listdir(base_render_dir)]
len(object_directories)


In [ ]:
object_directories

In [ ]:
import os
import shutil
import json
from tqdm import tqdm

base_render_dir = "/mnt/damian/.objaverse/CarCaptionData_ai_toolkit/" # "/mnt/damian/data/meshfleetXL/train/renders_eval_70" #  "/mnt/damian/Projects/diffusers_finetune/renders_eval/"
object_directories = [f"{base_render_dir}/{object_id}" for object_id in os.listdir(base_render_dir)]

dataset_goal_dir = base_render_dir # "/mnt/damian/.objaverse/CarCaptionData_ai_toolkit"
os.makedirs(dataset_goal_dir, exist_ok=True)

list_for_metadata_jsonl = []
for render_file in tqdm(object_directories):
    if render_file.endswith(".png"):
        # object is the filname without the extension
        new_file_name = os.path.basename(render_file)
        caption_file_name = f"{new_file_name.replace('.png', '.txt')}" 
        if os.path.exists(f"{dataset_goal_dir}/{caption_file_name}"):
            print(f"File {new_file_name} already exists, skipping.")
            continue
        caption = create_image_caption(render_file, pipe)
        # create the txt file in the goal directory
        with open(f"{dataset_goal_dir}/{caption_file_name}", "w") as f:
            f.write(caption)
        image_dict = {"file_name": new_file_name,"prompt": caption}
        list_for_metadata_jsonl.append({"file_name": new_file_name, "prompt": caption})

with open(f"{dataset_goal_dir}/metadata.jsonl", "w") as f:
    for item in list_for_metadata_jsonl:
        f.write(json.dumps(item) + "\n")

In [ ]:
import os
import shutil
import json
from tqdm import tqdm

base_render_dir = "/mnt/damian/.objaverse/CarCaptionData_ai_toolkit/" # "/mnt/damian/data/meshfleetXL/train/renders_eval_70" #  "/mnt/damian/Projects/diffusers_finetune/renders_eval/"
object_directories = [f"{base_render_dir}/{object_id}" for object_id in os.listdir(base_render_dir)]

dataset_goal_dir = "/mnt/damian/.objaverse/CarCaptionData_ai_toolkit_captions"
os.makedirs(dataset_goal_dir, exist_ok=True)
images_goal_dir = f"{dataset_goal_dir}/train"
os.makedirs(images_goal_dir, exist_ok=True)
jsonl_goal_dir = f"{dataset_goal_dir}/captions"
os.makedirs(jsonl_goal_dir, exist_ok=True)

list_for_metadata_jsonl = []
for object_dir in tqdm(object_directories):
    object_id = os.path.basename(object_dir)
    for render_file in os.listdir(object_dir):
        if render_file.endswith(".png"):
            new_file_name = f"{object_id}_{render_file}"
            json_file_name = f"{new_file_name.replace('.png', '.json')}" 
            if os.path.exists(f"{images_goal_dir}/{new_file_name}") and os.path.exists(f"{jsonl_goal_dir}/{json_file_name}"):
                dict_from_json_file = json.load(open(f"{jsonl_goal_dir}/{json_file_name}"))
                list_for_metadata_jsonl.append(dict_from_json_file)
                continue
            shutil.copy(f"{object_dir}/{render_file}", f"{images_goal_dir}/{new_file_name}")
            caption = create_image_caption(f"{object_dir}/{render_file}", pipe)
            image_dict = {"file_name": new_file_name,"prompt": caption}
            with open(f"{jsonl_goal_dir}/{json_file_name}", "w") as f:
                f.write(json.dumps(image_dict))
            list_for_metadata_jsonl.append({"file_name": new_file_name, "prompt": caption})

with open(f"{dataset_goal_dir}/metadata.jsonl", "w") as f:
    for item in list_for_metadata_jsonl:
        f.write(json.dumps(item) + "\n")

In [ ]:
list_for_metadata_jsonl